## A demo using Hydrogen Hamiltonian with GPT-QE.

In [44]:
import torch
from gqe.mingpt.utils import set_seed

set_seed(3407)

In [45]:
from qwrapper.operator import PauliObservable
from gqe.mingpt.cost import EnergyCost
from qswift.compiler import DefaultOperatorPool
from benchmark.molecule import DiatomicMolecularHamiltonian
from gqe.operator_pool.uccsd import UCCSD, do_generate_molecule
from gqe.common.initializer import HFStateInitializer
from gqe.util import get_device
from gqe.mingpt.callback import DefaultCallback, PretrainMonitor, PrintMonitor, FileMonitor

# molecule = generate_molecule("Li", "H", 1.596, "sto-3g", bravyi_kitaev=False)
bond_length = 3.0
geometry = f"H 0.0 0.0 0.0\n" + f"Be 0.0 0.0 {bond_length}\n" + f"H 0.0 0.0 {2 * bond_length}\n"
molecule = do_generate_molecule(geometry, "sto-3g", bravyi_kitaev=False)
nqubit = 12

# prepare Hamiltonian
hamiltonian = DiatomicMolecularHamiltonian(nqubit, molecule, bravyi_kitaev=False)

# prepare operator_pool
uccsd = UCCSD(nqubit, molecule)
paulis = uccsd.paulis
paulis.append(PauliObservable("IIIIIIIIII"))
print('paulis', paulis)
num_operators = len(paulis)
initializer = HFStateInitializer(n_electrons=4)
pool = DefaultOperatorPool(paulis)
cost = EnergyCost(hamiltonian, initializer, pool,
                  [1 / 320, -1 / 320, 1 / 160, -1 / 160, 1 / 80, -1 / 80, 1 / 40, -1 / 40, 0.05, -0.05, 0.1, -0.1, 0.2,
                   -0.2])


converged SCF energy = -15.0242100060364
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
paulis [+IIIYZXIIIIII, +IIIYZZZXIIII, +IIIYZZZZZXII, +IIIYZZZZZZZX, +IIXXIIIIIIYX, +IIXXIIIIYXII, +IIXXIIYXIIII, +IIXXYXIIIIII,

## FCI energy by diagonalization

In [46]:
from qwrapper.hamiltonian import compute_ground_state

#print(compute_ground_state(hamiltonian))

In [47]:
print("hf state:", hamiltonian.exact_value(initializer.init_circuit(12, [], "qulacs")))

hf state: -15.024210006036476


## Setup for GPT

In [48]:
# create a GPT instance
from gqe.mingpt.model import GPT


def get_gpt(n_gates):
    model_config = GPT.get_default_config()
    model_config.model_type = 'gpt2'
    model_config.vocab_size = cost.vocab_size()
    model_config.n_gates = n_gates  # The number of gates for each circuit
    model_config.block_size = model_config.n_gates
    model_config.temperature = 5  # Each gate is generated with probability exp(-temperature * logit)
    model_config.embd_pdrop = 0
    model_config.resid_pdrop = 0
    model_config.attn_pdrop = 0
    model_config.std = 0.02
    model_config.energy_offset = 14
    model_config.embd_pdrop = 0
    model_config.resid_pdrop = 0
    model_config.attn_pdrop = 0
    return GPT(model_config, cost)


from gqe.mingpt.trainer import Trainer

train_config = Trainer.get_default_config()
train_config.learning_rate = 5e-7  # the model we're using is so small that we can go a bit faster
train_config.max_iters = 3
train_config.num_workers = 10
train_config.n_samples = 100


In [51]:
from torch.utils.data import DataLoader
from gqe.mingpt.data import EnergyDataset

# print(files)
seed = 31
temperature = 100
n_gates = 12
model = get_gpt(n_gates)
model.load_state_dict(torch.load(f'../saved_models/gptqe_train_beh2_{n_gates}_{seed}_50'))
set_seed(seed)
file_monitor = FileMonitor()
train_config.max_iters = 200
train_config.n_samples = 50
model.temperature = temperature
train_config.learning_rate = 5e-7
trainer = Trainer(train_config, model)
trainer.set_callback('on_batch_end', DefaultCallback(model, monitors=[PrintMonitor(), file_monitor],
                                                     del_temperature=0.01).generate())
file_name = f"../output/trajectory_beh2_{n_gates}_{seed}_{temperature}.json"
trainer.run()
torch.save(model.state_dict(), f'../saved_models/gptqe_train_beh2_{n_gates}_{seed}_{temperature}')
file_monitor.save(file_name)
# files.append(file_name)

number of parameters: 85.89M
running on device mps
iter_dt 0.00s; iter 0: train loss 0.00179 temperature: 100
mean_logits tensor([-15.2740, -15.2824, -15.2714, -15.2725, -15.2750, -15.2824, -15.2799,
        -15.2794, -15.2817, -15.2726, -15.2767, -15.2761, -15.2808, -15.2827,
        -15.2772, -15.2716, -15.2768, -15.2799, -15.2753, -15.2825, -15.2813,
        -15.2800, -15.2753, -15.2816, -15.2682, -15.2850, -15.2801, -15.2782,
        -15.2771, -15.2797, -15.2829, -15.2769, -15.2756, -15.2813, -15.2800,
        -15.2712, -15.2771, -15.2855, -15.2764, -15.2800, -15.2754, -15.2802,
        -15.2705, -15.2849, -15.2739, -15.2753, -15.2877, -15.2858, -15.2772,
        -15.2790], device='mps:0', grad_fn=<SubBackward0>)
energies: tensor([-15.2452, -15.2740, -15.2650, -15.2631, -15.2761, -15.2631, -15.2692,
        -15.2647, -15.2698, -15.2781, -15.2660, -15.2661, -15.2731, -15.2714,
        -15.2751, -15.2660, -15.2706, -15.2621, -15.2694, -15.2691, -15.2629,
        -15.2744, -15.2739, -

KeyboardInterrupt: 

In [ ]:
# file_name = f"../output/trajectory_beh2_{n_gates}_{seed}_{temperature}.json"
# torch.save(model.state_dict(), f'../saved_models/gptqe_train_beh2_{n_gates}_{seed}_{temperature}')
# file_monitor.save(file_name)